In [1]:
!pip install pyspark surprise

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.0/317.0 MB 3.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 772.0/772.0 kB 63.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.1-py2.py3-none-any.whl size=317488491 sha256=0a047dd060a52d84ab42bf77b46011d2a680de55711090a5c26c3725eab6a5ba
  Stored in directory: /root/.cache/pip/wheels/80/1d/60/2c256ed38dddce2fdd93be545214a63e02fbd8d74fb0b7f3a6
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.3-cp310-cp310-linux_x86_64.whl size=3163006 sha256=9722ee257093f8047d39ae2909fd90368280a316651a0f297d63fb81568324b1
  Stored in directory: /root/.cache/pip/wheels/a5/ca/a8/4e28def53797fdc4363ca4af740db15a9c2f1595ebc51fb445
Successfully built pyspark scikit-surprise


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pyspark


import warnings
warnings.filterwarnings('ignore')

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [198]:
DATA_DIR = '/content/drive/MyDrive/Final_DS5230'

In [199]:
df = pd.read_csv(DATA_DIR + '/df_with_metrics.csv')

In [200]:
df.head()

,game_id,id,language,started_at,title,type,user_id,user_name,viewer_count,user_login,game_name,tags,is_mature,box_art_url,genre,ended_at,duration_minutes,engagement_score,normalized_engagement_score,diversity_score
0,766571430,44080159291,en,2024-04-23 12:37:09+00:00,🛑 Let's Have A Beautiful Day 🛑 !Ghost !Podcast,live,41652,Merk,229,merk,HELLDIVERS 2,"['English', 'DropsEnabled']",True,https://static-cdn.jtvnw.net/ttv-boxart/766571...,Indie,2024-04-23 18:33:40.767189+00:00,356.53,81645.37,0.003625,0.92
1,766571430,44080159291,en,2024-04-23 12:37:09+00:00,🛑 Let's Have A Beautiful Day 🛑 !Ghost !Podcast,live,41652,Merk,229,merk,HELLDIVERS 2,"['English', 'DropsEnabled']",True,https://static-cdn.jtvnw.net/ttv-boxart/766571...,Shooter,2024-04-23 18:33:40.767189+00:00,356.53,81645.37,0.003625,0.92
2,509673,50945912893,en,2024-04-23 03:44:48+00:00,"Welcome to the 24/7 Geek, Nerd, Retro, Tech, &...",live,61127,ChrisPirillo,8,chrispirillo,Makers & Crafting,"['starwars', 'tech', 'videogames', 'toys', 'ar...",False,https://static-cdn.jtvnw.net/ttv-boxart/509673...,NaN,2024-04-23 18:33:40.767189+00:00,888.88,7111.04,0.000316,1.00
3,489776,40577448373,en,2024-04-23 15:32:38+00:00,Fallout 4 - Power Armor Boi -> Hey look! Henry...,live,71336,Robbaz,1159,robbaz,Fallout 4,['English'],True,https://static-cdn.jtvnw.net/ttv-boxart/489776...,Shooter,2024-04-23 18:33:40.767189+00:00,181.05,209836.95,0.009318,0.92
4,489776,40577448373,en,2024-04-23 15:32:38+00:00,Fallout 4 - Power Armor Boi -> Hey look! Henry...,live,71336,Robbaz,1159,robbaz,Fallout 4,['English'],True,https://static-cdn.jtvnw.net/ttv-boxart/489776...,Role-playing (RPG),2024-04-23 18:33:40.767189+00:00,181.05,209836.95,0.009318,0.92


In [163]:
df.isna().sum()

game_id                           0
id                                0
language                          0
started_at                        0
title                           252
type                              0
user_id                           0
user_name                         0
viewer_count                      0
user_login                        0
game_name                         0
tags                            149
is_mature                         0
box_art_url                       0
genre                          4287
ended_at                          0
duration_minutes                  0
engagement_score                  0
normalized_engagement_score       0
diversity_score                   0
dtype: int64

In [164]:
df.dropna(inplace=True)

In [165]:
df.isna().sum()

game_id                        0
id                             0
language                       0
started_at                     0
title                          0
type                           0
user_id                        0
user_name                      0
viewer_count                   0
user_login                     0
game_name                      0
tags                           0
is_mature                      0
box_art_url                    0
genre                          0
ended_at                       0
duration_minutes               0
engagement_score               0
normalized_engagement_score    0
diversity_score                0
dtype: int64

In [166]:
df.shape

(69312, 20)

In [167]:
df.engagement_score.describe()

count    6.931200e+04
mean     2.699486e+04
std      2.881623e+05
min      0.000000e+00
25%      2.546600e+02
50%      8.649900e+02
75%      3.820740e+03
max      2.251976e+07
Name: engagement_score, dtype: float64

In [168]:
df = df[df.engagement_score != 0]

In [169]:
df.drop(columns=['tags', 'language'], inplace=True)

In [170]:
df

,game_id,id,started_at,title,type,user_id,user_name,viewer_count,user_login,game_name,is_mature,box_art_url,genre,ended_at,duration_minutes,engagement_score,normalized_engagement_score,diversity_score
0,766571430,44080159291,2024-04-23 12:37:09+00:00,🛑 Let's Have A Beautiful Day 🛑 !Ghost !Podcast,live,41652,Merk,229,merk,HELLDIVERS 2,True,https://static-cdn.jtvnw.net/ttv-boxart/766571...,Indie,2024-04-23 18:33:40.767189+00:00,356.53,81645.37,0.003625,0.92
1,766571430,44080159291,2024-04-23 12:37:09+00:00,🛑 Let's Have A Beautiful Day 🛑 !Ghost !Podcast,live,41652,Merk,229,merk,HELLDIVERS 2,True,https://static-cdn.jtvnw.net/ttv-boxart/766571...,Shooter,2024-04-23 18:33:40.767189+00:00,356.53,81645.37,0.003625,0.92
3,489776,40577448373,2024-04-23 15:32:38+00:00,Fallout 4 - Power Armor Boi -> Hey look! Henry...,live,71336,Robbaz,1159,robbaz,Fallout 4,True,https://static-cdn.jtvnw.net/ttv-boxart/489776...,Shooter,2024-04-23 18:33:40.767189+00:00,181.05,209836.95,0.009318,0.92
4,489776,40577448373,2024-04-23 15:32:38+00:00,Fallout 4 - Power Armor Boi -> Hey look! Henry...,live,71336,Robbaz,1159,robbaz,Fallout 4,True,https://static-cdn.jtvnw.net/ttv-boxart/489776...,Role-playing (RPG),2024-04-23 18:33:40.767189+00:00,181.05,209836.95,0.009318,0.92
5,766571430,50947453741,2024-04-23 12:55:51+00:00,"TOP - Democracy, I just love it.",live,161841,Mireille_Mathieu,8,mireille_mathieu,HELLDIVERS 2,True,https://static-cdn.jtvnw.net/ttv-boxart/766571...,Indie,2024-04-23 18:33:40.767189+00:00,337.83,2702.64,0.000120,0.92
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
73971,279014883,42247246840,2024-04-23 16:52:35+00:00,Хацрне Мику💙,live,1070869266,liks_105,1,liks_105,Hatsune Miku: Colorful Stage!,False,https://static-cdn.jtvnw.net/ttv-boxart/279014...,Music,2024-04-23 18:33:40.767189+00:00,101.10,101.10,0.000004,0.88
73972,279014883,42247246840,2024-04-23 16:52:35+00:00,Хацрне Мику💙,live,1070869266,liks_105,1,liks_105,Hatsune Miku: Colorful Stage!,False,https://static-cdn.jtvnw.net/ttv-boxart/279014...,Adventure,2024-04-23 18:33:40.767189+00:00,101.10,101.10,0.000004,0.88
73973,279014883,42247246840,2024-04-23 16:52:35+00:00,Хацрне Мику💙,live,1070869266,liks_105,1,liks_105,Hatsune Miku: Colorful Stage!,False,https://static-cdn.jtvnw.net/ttv-boxart/279014...,Visual Novel,2024-04-23 18:33:40.767189+00:00,101.10,101.10,0.000004,0.88
73974,513501,42247193320,2024-04-23 16:45:04+00:00,Го востанавливать скилл ( не играл 2 года ),live,1070872089,dalbayacherua,1,dalbayacherua,Standoff 2,False,https://static-cdn.jtvnw.net/ttv-boxart/513501...,Shooter,2024-04-23 18:33:40.767189+00:00,108.61,108.61,0.000005,0.96


In [171]:
def calculate_combined_metric(engagement_score, diversity_score, alpha=0.7):
    """
    Calculate a combined metric that balances engagement and diversity.

    Parameters:
    - engagement_score (float): The engagement score, calculated as the product of viewer count and duration.
    - diversity_score (float): The diversity score, calculated from genre data.
    - alpha (float): The weight given to the engagement score in the combined metric.
                     The diversity score is weighted as (1 - alpha).

    Returns:
    - combined_score (float): The combined score reflecting both engagement and diversity.
    """
    # Normalize engagement score if needed
    # Here we assume engagement score ranges are already scaled properly
    # If not, include normalization logic here.

    # Normalize diversity score if needed
    # Diversity score should be between 0 and 1, adjust if the range differs.

    combined_score = alpha * engagement_score + (1 - alpha) * diversity_score
    return combined_score

In [172]:
df['combined_score'] = df.apply(lambda row: calculate_combined_metric(row['engagement_score'], row['diversity_score']), axis=1)

In [173]:
df.columns

Index(['game_id', 'id', 'started_at', 'title', 'type', 'user_id', 'user_name',
       'viewer_count', 'user_login', 'game_name', 'is_mature', 'box_art_url',
       'genre', 'ended_at', 'duration_minutes', 'engagement_score',
       'normalized_engagement_score', 'diversity_score', 'combined_score'],
      dtype='object')

In [18]:
from sklearn.decomposition import TruncatedSVD

# Assuming df is your DataFrame
matrix = df.pivot_table(index='user_id', columns='id', values='engagement_score', fill_value=0)
svd = TruncatedSVD(n_components=20)
user_features = svd.fit_transform(matrix)
streamer_features = svd.components_.T

# Predict scores for all user-streamer pairs
predicted_scores = pd.DataFrame(user_features.dot(streamer_features.T), index=matrix.index, columns=matrix.columns)

# For each user, select top N streamers as candidates
top_candidates = predicted_scores.apply(lambda x: x.nlargest(10).index.tolist(), axis=1)

In [174]:
df['interaction_score'] = (df['viewer_count'] * df['duration_minutes'] * df['engagement_score']).pow(1/3)  # Geometric mean for normalization

In [175]:
df.head()

,game_id,id,started_at,title,type,user_id,user_name,viewer_count,user_login,game_name,is_mature,box_art_url,genre,ended_at,duration_minutes,engagement_score,normalized_engagement_score,diversity_score,combined_score,interaction_score
0,766571430,44080159291,2024-04-23 12:37:09+00:00,🛑 Let's Have A Beautiful Day 🛑 !Ghost !Podcast,live,41652,Merk,229,merk,HELLDIVERS 2,True,https://static-cdn.jtvnw.net/ttv-boxart/766571...,Indie,2024-04-23 18:33:40.767189+00:00,356.53,81645.37,0.003625,0.92,57152.035,1882.006162
1,766571430,44080159291,2024-04-23 12:37:09+00:00,🛑 Let's Have A Beautiful Day 🛑 !Ghost !Podcast,live,41652,Merk,229,merk,HELLDIVERS 2,True,https://static-cdn.jtvnw.net/ttv-boxart/766571...,Shooter,2024-04-23 18:33:40.767189+00:00,356.53,81645.37,0.003625,0.92,57152.035,1882.006162
3,489776,40577448373,2024-04-23 15:32:38+00:00,Fallout 4 - Power Armor Boi -> Hey look! Henry...,live,71336,Robbaz,1159,robbaz,Fallout 4,True,https://static-cdn.jtvnw.net/ttv-boxart/489776...,Shooter,2024-04-23 18:33:40.767189+00:00,181.05,209836.95,0.009318,0.92,146886.141,3531.191822
4,489776,40577448373,2024-04-23 15:32:38+00:00,Fallout 4 - Power Armor Boi -> Hey look! Henry...,live,71336,Robbaz,1159,robbaz,Fallout 4,True,https://static-cdn.jtvnw.net/ttv-boxart/489776...,Role-playing (RPG),2024-04-23 18:33:40.767189+00:00,181.05,209836.95,0.009318,0.92,146886.141,3531.191822
5,766571430,50947453741,2024-04-23 12:55:51+00:00,"TOP - Democracy, I just love it.",live,161841,Mireille_Mathieu,8,mireille_mathieu,HELLDIVERS 2,True,https://static-cdn.jtvnw.net/ttv-boxart/766571...,Indie,2024-04-23 18:33:40.767189+00:00,337.83,2702.64,0.000120,0.92,1892.124,194.025495


In [176]:
df.interaction_score.describe()

count    67822.000000
mean       430.871873
std       1682.072136
min         20.570954
25%         42.033826
50%         94.223767
75%        251.110747
max      79746.056619
Name: interaction_score, dtype: float64

In [21]:
df[df.id == 42544331033]

,game_id,id,started_at,title,type,user_id,user_name,viewer_count,user_login,game_name,is_mature,box_art_url,genre,ended_at,duration_minutes,engagement_score,normalized_engagement_score,diversity_score
5625,21779,42544331033,2024-04-23 09:46:13+00:00,LOL COM !tipspace,live,47071880,YoDa,7954,yoda,League of Legends,False,https://static-cdn.jtvnw.net/ttv-boxart/21779-...,Role-playing (RPG),2024-04-23 18:33:40.767189+00:00,527.46,4195416.84,0.186299,0.88
5626,21779,42544331033,2024-04-23 09:46:13+00:00,LOL COM !tipspace,live,47071880,YoDa,7954,yoda,League of Legends,False,https://static-cdn.jtvnw.net/ttv-boxart/21779-...,Strategy,2024-04-23 18:33:40.767189+00:00,527.46,4195416.84,0.186299,0.88
5627,21779,42544331033,2024-04-23 09:46:13+00:00,LOL COM !tipspace,live,47071880,YoDa,7954,yoda,League of Legends,False,https://static-cdn.jtvnw.net/ttv-boxart/21779-...,MOBA,2024-04-23 18:33:40.767189+00:00,527.46,4195416.84,0.186299,0.88


In [30]:
from pyspark.sql import SparkSession
from pyspark.ml.recommendation import ALS

spark = SparkSession.builder.appName("Streamer Recommender").getOrCreate()

In [32]:
df.shape

(67822, 20)

In [33]:
sdf = spark.createDataFrame(df)

In [34]:
# Show the DataFrame to confirm
sdf.show()

+----------+-----------+--------------------+--------------------+----+-------+----------------+------------+----------------+--------------------+---------+--------------------+--------------------+--------------------+----------------+----------------+---------------------------+---------------+------------------+------------------+
|   game_id|         id|          started_at|               title|type|user_id|       user_name|viewer_count|      user_login|           game_name|is_mature|         box_art_url|               genre|            ended_at|duration_minutes|engagement_score|normalized_engagement_score|diversity_score| interaction_score|    combined_score|
+----------+-----------+--------------------+--------------------+----+-------+----------------+------------+----------------+--------------------+---------+--------------------+--------------------+--------------------+----------------+----------------+---------------------------+---------------+------------------+---------

In [81]:
# Load data
data = sdf.select('user_id', 'game_id', 'interaction_score')

# ALS model requires numeric values
data = data.na.fill(0)  # Handling missing values

# ALS model setup
als = ALS(maxIter=10, regParam=0.03, userCol="user_id", itemCol="game_id", ratingCol="interaction_score", implicitPrefs=True)


In [82]:
data.show()

+-------+----------+------------------+
|user_id|   game_id| interaction_score|
+-------+----------+------------------+
|  41652| 766571430|1882.0061618320553|
|  41652| 766571430|1882.0061618320553|
|  71336|    489776|3531.1918224510787|
|  71336|    489776|3531.1918224510787|
| 161841| 766571430|194.02549501617102|
| 161841| 766571430|194.02549501617102|
| 265389|    492287|114.30219710693424|
| 265389|    492287|114.30219710693424|
| 265389|    492287|114.30219710693424|
| 265389|    492287|114.30219710693424|
| 278060|1003238276|42.891163373202204|
| 278060|1003238276|42.891163373202204|
| 278060|1003238276|42.891163373202204|
| 346492| 468880140| 53.22084513934939|
| 346492| 468880140| 53.22084513934939|
| 346492| 468880140| 53.22084513934939|
| 501329|1778360343| 67.45787238707061|
| 501329|1778360343| 67.45787238707061|
| 506767| 538054672|229.20441947564956|
| 507125| 407922769|  66.3158416297654|
+-------+----------+------------------+
only showing top 20 rows



In [83]:
model = als.fit(data)

# Recommend games for all users
game_recommendations = model.recommendForAllUsers(10)

In [84]:
game_recommendations.show()

+--------+--------------------+
| user_id|     recommendations|
+--------+--------------------+
| 2760283|[{8259, 0.4354016...|
| 7243387|[{32399, 0.976903...|
| 8320445|[{33214, 1.338264...|
|11308729|[{745589378, 0.68...|
|17331543|[{513143, 1.15522...|
|18076161|[{512818, 1.09637...|
|20386324|[{32982, 0.481528...|
|20405964|[{389876637, 0.99...|
|20929454|[{27284, 0.924186...|
|21100325|[{18763, 0.889312...|
|21805913|[{518873, 0.73802...|
|25105883|[{65654, 0.804296...|
|25254906|[{506416, 0.7498}...|
|25567160|[{55453844, 0.997...|
|25677980|[{32982, 0.999422...|
|26290555|[{687129551, 0.99...|
|26348106|[{987123848, 0.97...|
|26969086|[{478407715, 0.66...|
|27195973|[{890558948, 0.71...|
|27201220|[{24241, 0.8619},...|
+--------+--------------------+
only showing top 20 rows



In [86]:
recommendation_df = game_recommendations.toPandas()

In [87]:
recommendation_df.head(20)

,user_id,recommendations
0,2760283,"[(8259, 0.43540167808532715), (115977, 0.37942..."
1,7243387,"[(32399, 0.9769031405448914), (1185403782, 0.7..."
2,8320445,"[(33214, 1.3382645845413208), (519103, 1.13155..."
3,11308729,"[(745589378, 0.6811476945877075), (212742304, ..."
4,17331543,"[(513143, 1.15522301197052), (493959, 1.110783..."
5,18076161,"[(512818, 1.0963711738586426), (386821, 1.0654..."
6,20386324,"[(32982, 0.4815281927585602), (497497, 0.45311..."
7,20405964,"[(389876637, 0.9970801472663879), (32982, 0.80..."
8,20929454,"[(27284, 0.9241869449615479), (18122, 0.749787..."
9,21100325,"[(18763, 0.8893126845359802), (138585, 0.77696..."


In [99]:
for user_id in recommendation_df['user_id'].unique()[:10]:
    user_recommendations = recommendation_df[recommendation_df['user_id'] == user_id]
    print(user_id)
    print(user_recommendations["recommendations"].values)
    print()

2760283
[list([Row(game_id=8259, rating=0.43540167808532715), Row(game_id=115977, rating=0.37942010164260864), Row(game_id=33214, rating=0.3763173818588257), Row(game_id=494184, rating=0.37604039907455444), Row(game_id=518901, rating=0.3416852056980133), Row(game_id=19554, rating=0.3361012637615204), Row(game_id=24241, rating=0.3325754404067993), Row(game_id=65632, rating=0.3320464789867401), Row(game_id=66366, rating=0.32367074489593506), Row(game_id=13263, rating=0.2995813190937042)])]

7243387
[list([Row(game_id=32399, rating=0.9769031405448914), Row(game_id=1185403782, rating=0.7246028780937195), Row(game_id=29307, rating=0.6841143369674683), Row(game_id=497497, rating=0.6797577142715454), Row(game_id=1470211036, rating=0.6352933049201965), Row(game_id=23020, rating=0.6325437426567078), Row(game_id=2106425216, rating=0.5205073356628418), Row(game_id=504735, rating=0.5063906908035278), Row(game_id=518015, rating=0.4951269030570984), Row(game_id=490422, rating=0.46515485644340515)])]

In [102]:
df[df.game_id == 33214 ]


,game_id,id,started_at,title,type,user_id,user_name,viewer_count,user_login,game_name,is_mature,box_art_url,genre,ended_at,duration_minutes,engagement_score,normalized_engagement_score,diversity_score,interaction_score,combined_score
141,33214,42545033241,2024-04-23 15:49:38+00:00,[EN/PS5] Zero Build Squads | @master_of_chaos ...,live,6936526,Master_Of_Chaos,148,master_of_chaos,Fortnite,False,https://static-cdn.jtvnw.net/ttv-boxart/33214-...,Shooter,2024-04-23 18:33:40.767189+00:00,164.05,24279.40,0.001078,0.96,838.478569,16995.868
553,33214,50947475981,2024-04-23 13:00:00+00:00,NEW FORTNITE UPDATE LETS GET IT,live,19571641,Ninja,9174,ninja,Fortnite,False,https://static-cdn.jtvnw.net/ttv-boxart/33214-...,Shooter,2024-04-23 18:33:40.767189+00:00,333.68,3061180.32,0.135933,0.96,21082.686059,2142826.512
704,33214,42544750505,2024-04-23 14:07:28+00:00,Zero Build Solos! !merch | Use Code: Prosperin...,live,22160589,Prospering,671,prospering,Fortnite,False,https://static-cdn.jtvnw.net/ttv-boxart/33214-...,Shooter,2024-04-23 18:33:40.767189+00:00,266.21,178626.91,0.007932,0.96,3171.742516,125039.125
1702,33214,42247097912,2024-04-23 16:29:46+00:00,UPDATE! KEBAB PROP HUNT *1822-1963-6181* !give...,live,29356850,keebabb,100,keebabb,Fortnite,False,https://static-cdn.jtvnw.net/ttv-boxart/33214-...,Shooter,2024-04-23 18:33:40.767189+00:00,123.91,12391.00,0.000550,0.96,535.472992,8673.988
1742,33214,50947745213,2024-04-23 13:50:16+00:00,🔴 NUEVO FUSIL | !redes @Dheylo,live,29530640,Dheylo,675,dheylo,Fortnite,False,https://static-cdn.jtvnw.net/ttv-boxart/33214-...,Shooter,2024-04-23 18:33:40.767189+00:00,283.41,191301.75,0.008495,0.96,3320.060226,133911.513
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65184,33214,40577316645,2024-04-23 14:49:27+00:00,🔥 YUHUUU Tatsu wieder live🔥 | !turnier !dc !yt...,live,899706470,McTatsu,117,mctatsu,Fortnite,False,https://static-cdn.jtvnw.net/ttv-boxart/33214-...,Shooter,2024-04-23 18:33:40.767189+00:00,224.23,26234.91,0.001165,0.96,882.916675,18364.725
65224,33214,40577480245,2024-04-23 15:42:17+00:00,Herz Willkommen in meinem Stream!,live,900709623,Shaila318,226,shaila318,Fortnite,False,https://static-cdn.jtvnw.net/ttv-boxart/33214-...,Shooter,2024-04-23 18:33:40.767189+00:00,171.40,38736.40,0.001720,0.96,1144.843628,27115.768
66265,33214,40577621157,2024-04-23 16:27:56+00:00,EVAL CUP W TEUZZ!! | !map | !merch,live,924287633,fragger_sandropt,389,fragger_sandropt,Fortnite,True,https://static-cdn.jtvnw.net/ttv-boxart/33214-...,Shooter,2024-04-23 18:33:40.767189+00:00,125.75,48916.75,0.002172,0.96,1337.534717,34242.013
68667,33214,50948250861,2024-04-23 15:01:52+00:00,🚨GRINDING 🚨EVAL CUP AT 19:00🚨QUALED LOWER BRAC...,live,984432588,fnmoneymaker,1038,fnmoneymaker,Fortnite,False,https://static-cdn.jtvnw.net/ttv-boxart/33214-...,Shooter,2024-04-23 18:33:40.767189+00:00,211.81,219858.78,0.009763,0.96,3642.748677,153901.434


In [104]:
from pyspark.sql.window import Window
from pyspark.ml.recommendation import ALS
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.feature import StringIndexer
from pyspark.ml.tuning import TrainValidationSplit, ParamGridBuilder, CrossValidator

In [105]:
ALS(userCol="user_id", itemCol="game_id", ratingCol="interaction_score", implicitPrefs=True)

ALS_0967ddec9010

In [112]:
data.show()

+-------+----------+------------------+
|user_id|   game_id| interaction_score|
+-------+----------+------------------+
|  41652| 766571430|1882.0061618320553|
|  41652| 766571430|1882.0061618320553|
|  71336|    489776|3531.1918224510787|
|  71336|    489776|3531.1918224510787|
| 161841| 766571430|194.02549501617102|
| 161841| 766571430|194.02549501617102|
| 265389|    492287|114.30219710693424|
| 265389|    492287|114.30219710693424|
| 265389|    492287|114.30219710693424|
| 265389|    492287|114.30219710693424|
| 278060|1003238276|42.891163373202204|
| 278060|1003238276|42.891163373202204|
| 278060|1003238276|42.891163373202204|
| 346492| 468880140| 53.22084513934939|
| 346492| 468880140| 53.22084513934939|
| 346492| 468880140| 53.22084513934939|
| 501329|1778360343| 67.45787238707061|
| 501329|1778360343| 67.45787238707061|
| 506767| 538054672|229.20441947564956|
| 507125| 407922769|  66.3158416297654|
+-------+----------+------------------+
only showing top 20 rows



In [113]:
data.count()

67822

In [114]:
train, test = data.randomSplit([0.7,0.3],67822)
train.cache()
test.cache()

DataFrame[user_id: bigint, game_id: bigint, interaction_score: double]

In [115]:
als = ALS(userCol='user_id', itemCol='game_id', ratingCol='interaction_score',
          coldStartStrategy='drop', nonnegative=True)


param_grid = ParamGridBuilder()\
             .addGrid(als.rank, [10, 100])\
             .addGrid(als.maxIter, [10])\
             .addGrid(als.regParam, [.05, .1, .15])\
             .build()
evaluator = RegressionEvaluator(metricName='rmse', labelCol='interaction_score', predictionCol='prediction')

cv = CrossValidator(estimator=als, estimatorParamMaps=param_grid, evaluator=evaluator, numFolds=3, parallelism = 6)



model = cv.fit(train)

In [116]:
best_model = model.bestModel

print(f"Rank = {best_model._java_obj.parent().getRank()}")
print(f"MaxIter = {best_model._java_obj.parent().getMaxIter()}")
print(f"RegParam = {best_model._java_obj.parent().getRegParam()}")

Rank = 10
MaxIter = 10
RegParam = 0.15


In [117]:
prediction = best_model.transform(test)
rmse = evaluator.evaluate(prediction)
print(f'RMSE = {rmse}')

# we can get the user latent factors and item latent factors from the model
best_model.userFactors.show()
best_model.itemFactors.show()

RMSE = 923.8442840798995
+--------+--------------------+
|      id|            features|
+--------+--------------------+
|  278060|[0.061853983, 0.1...|
| 2400710|[0.0, 0.0, 6.6201...|
| 3028810|[0.0, 0.0, 0.1099...|
| 5367230|[0.0, 0.0, 4.4540...|
| 5838420|[0.5481643, 0.114...|
| 7162370|[2.0486722, 0.0, ...|
| 8919020|[0.0, 0.0, 0.0, 0...|
| 9094040|[0.0, 0.0, 0.0, 0...|
|13818680|[7.4364715, 0.0, ...|
|15054610|[0.93800056, 0.03...|
|16630260|[0.21741523, 0.0,...|
|17883810|[0.23725992, 0.10...|
|20270300|[7.016124E-4, 0.0...|
|20620820|[0.014255427, 0.1...|
|20694610|[2.589828, 0.0, 1...|
|21281250|[0.24383445, 0.03...|
|21390470|[0.0, 0.0, 0.0, 4...|
|21991090|[0.0, 0.78629756,...|
|22178840|[0.0, 0.0, 0.9657...|
|22885760|[1.3171611, 0.0, ...|
+--------+--------------------+
only showing top 20 rows

+----+--------------------+
|  id|            features|
+----+--------------------+
|1260|[16.683256, 0.0, ...|
|1270|[23.485502, 0.183...|
|1420|[211.95096, 310.8...|
|1520|[48.614

In [120]:
# Recommend games for all users
game_recommendations = best_model.recommendForAllUsers(10)

In [122]:
df.head()

,game_id,id,started_at,title,type,user_id,user_name,viewer_count,user_login,game_name,is_mature,box_art_url,genre,ended_at,duration_minutes,engagement_score,normalized_engagement_score,diversity_score,interaction_score,combined_score
0,766571430,44080159291,2024-04-23 12:37:09+00:00,🛑 Let's Have A Beautiful Day 🛑 !Ghost !Podcast,live,41652,Merk,229,merk,HELLDIVERS 2,True,https://static-cdn.jtvnw.net/ttv-boxart/766571...,Indie,2024-04-23 18:33:40.767189+00:00,356.53,81645.37,0.003625,0.92,1882.006162,57152.035
1,766571430,44080159291,2024-04-23 12:37:09+00:00,🛑 Let's Have A Beautiful Day 🛑 !Ghost !Podcast,live,41652,Merk,229,merk,HELLDIVERS 2,True,https://static-cdn.jtvnw.net/ttv-boxart/766571...,Shooter,2024-04-23 18:33:40.767189+00:00,356.53,81645.37,0.003625,0.92,1882.006162,57152.035
3,489776,40577448373,2024-04-23 15:32:38+00:00,Fallout 4 - Power Armor Boi -> Hey look! Henry...,live,71336,Robbaz,1159,robbaz,Fallout 4,True,https://static-cdn.jtvnw.net/ttv-boxart/489776...,Shooter,2024-04-23 18:33:40.767189+00:00,181.05,209836.95,0.009318,0.92,3531.191822,146886.141
4,489776,40577448373,2024-04-23 15:32:38+00:00,Fallout 4 - Power Armor Boi -> Hey look! Henry...,live,71336,Robbaz,1159,robbaz,Fallout 4,True,https://static-cdn.jtvnw.net/ttv-boxart/489776...,Role-playing (RPG),2024-04-23 18:33:40.767189+00:00,181.05,209836.95,0.009318,0.92,3531.191822,146886.141
5,766571430,50947453741,2024-04-23 12:55:51+00:00,"TOP - Democracy, I just love it.",live,161841,Mireille_Mathieu,8,mireille_mathieu,HELLDIVERS 2,True,https://static-cdn.jtvnw.net/ttv-boxart/766571...,Indie,2024-04-23 18:33:40.767189+00:00,337.83,2702.64,0.000120,0.92,194.025495,1892.124


In [177]:
from sklearn.preprocessing import StandardScaler

# Initialize the MinMaxScaler
scaler = StandardScaler()

# Fit and transform the column 'B'
df['interaction_score_normalized'] = scaler.fit_transform(df[['interaction_score']])

In [178]:
df.head()

,game_id,id,started_at,title,type,user_id,user_name,viewer_count,user_login,game_name,...,box_art_url,genre,ended_at,duration_minutes,engagement_score,normalized_engagement_score,diversity_score,combined_score,interaction_score,interaction_score_normalized
0,766571430,44080159291,2024-04-23 12:37:09+00:00,🛑 Let's Have A Beautiful Day 🛑 !Ghost !Podcast,live,41652,Merk,229,merk,HELLDIVERS 2,...,https://static-cdn.jtvnw.net/ttv-boxart/766571...,Indie,2024-04-23 18:33:40.767189+00:00,356.53,81645.37,0.003625,0.92,57152.035,1882.006162,0.862713
1,766571430,44080159291,2024-04-23 12:37:09+00:00,🛑 Let's Have A Beautiful Day 🛑 !Ghost !Podcast,live,41652,Merk,229,merk,HELLDIVERS 2,...,https://static-cdn.jtvnw.net/ttv-boxart/766571...,Shooter,2024-04-23 18:33:40.767189+00:00,356.53,81645.37,0.003625,0.92,57152.035,1882.006162,0.862713
3,489776,40577448373,2024-04-23 15:32:38+00:00,Fallout 4 - Power Armor Boi -> Hey look! Henry...,live,71336,Robbaz,1159,robbaz,Fallout 4,...,https://static-cdn.jtvnw.net/ttv-boxart/489776...,Shooter,2024-04-23 18:33:40.767189+00:00,181.05,209836.95,0.009318,0.92,146886.141,3531.191822,1.843169
4,489776,40577448373,2024-04-23 15:32:38+00:00,Fallout 4 - Power Armor Boi -> Hey look! Henry...,live,71336,Robbaz,1159,robbaz,Fallout 4,...,https://static-cdn.jtvnw.net/ttv-boxart/489776...,Role-playing (RPG),2024-04-23 18:33:40.767189+00:00,181.05,209836.95,0.009318,0.92,146886.141,3531.191822,1.843169
5,766571430,50947453741,2024-04-23 12:55:51+00:00,"TOP - Democracy, I just love it.",live,161841,Mireille_Mathieu,8,mireille_mathieu,HELLDIVERS 2,...,https://static-cdn.jtvnw.net/ttv-boxart/766571...,Indie,2024-04-23 18:33:40.767189+00:00,337.83,2702.64,0.000120,0.92,1892.124,194.025495,-0.140807


In [128]:
sdf = spark.createDataFrame(df)

In [129]:
ALS(userCol="user_id", itemCol="game_id", ratingCol="interaction_score_normalized", implicitPrefs=True)

ALS_9aba1be4b74e

In [130]:
sdf.count()

67822

In [131]:
data = sdf.select('user_id', 'game_id', 'interaction_score_normalized')

In [132]:
train, test = data.randomSplit([0.7,0.3],67822)
train.cache()
test.cache()

DataFrame[user_id: bigint, game_id: bigint, interaction_score_normalized: double]

In [133]:
als = ALS(userCol='user_id', itemCol='game_id', ratingCol='interaction_score_normalized',
          coldStartStrategy='drop', nonnegative=True)


param_grid = ParamGridBuilder()\
             .addGrid(als.rank, [10, 100])\
             .addGrid(als.maxIter, [10])\
             .addGrid(als.regParam, [.05, .1, .15])\
             .build()
evaluator = RegressionEvaluator(metricName='rmse', labelCol='interaction_score_normalized', predictionCol='prediction')

cv = CrossValidator(estimator=als, estimatorParamMaps=param_grid, evaluator=evaluator, numFolds=3, parallelism = 6)



model = cv.fit(train)

In [134]:
best_model = model.bestModel

print(f"Rank = {best_model._java_obj.parent().getRank()}")
print(f"MaxIter = {best_model._java_obj.parent().getMaxIter()}")
print(f"RegParam = {best_model._java_obj.parent().getRegParam()}")

Rank = 100
MaxIter = 10
RegParam = 0.05


In [135]:
prediction = best_model.transform(test)
rmse = evaluator.evaluate(prediction)
print(f'RMSE = {rmse}')

# we can get the user latent factors and item latent factors from the model
best_model.userFactors.show()
best_model.itemFactors.show()

RMSE = 0.18821269211331904
+--------+--------------------+
|      id|            features|
+--------+--------------------+
|  278060|[0.0, 0.0, 0.0, 0...|
| 2400710|[0.0, 0.086891994...|
| 3028810|[0.048889607, 0.0...|
| 5367230|[0.0, 0.0, 0.0, 0...|
| 5838420|[0.0, 0.0, 0.0, 0...|
| 7162370|[6.628581E-4, 0.0...|
| 8919020|[0.0, 0.0, 0.0, 0...|
| 9094040|[0.1276163, 0.0, ...|
|13818680|[0.09784774, 0.0,...|
|15054610|[0.0, 0.0, 0.0, 0...|
|16630260|[0.0718588, 0.043...|
|17883810|[0.0, 0.0, 0.0, 0...|
|20270300|[0.0, 0.0, 0.0, 0...|
|20620820|[0.0016408773, 0....|
|20694610|[0.0, 0.0, 0.0, 0...|
|21281250|[0.0, 0.0, 0.0, 0...|
|21390470|[0.17861778, 0.0,...|
|21991090|[0.1607066, 0.0, ...|
|22178840|[0.0, 0.0, 0.0, 0...|
|22885760|[0.0, 0.0, 0.0, 0...|
+--------+--------------------+
only showing top 20 rows

+----+--------------------+
|  id|            features|
+----+--------------------+
|1260|[0.0, 0.0, 0.0, 0...|
|1270|[0.0, 0.0, 0.0, 0...|
|1420|[0.0, 0.0, 0.0, 0...|
|1520|[0.0,

In [138]:
test.show()

+-------+----------+----------------------------+
|user_id|   game_id|interaction_score_normalized|
+-------+----------+----------------------------+
| 346492| 468880140|        -0.22451701325601905|
| 501329|1778360343|        -0.21605296947191327|
| 501329|1778360343|        -0.21605296947191327|
| 528871|     29595|           1.206522879017659|
| 649302|1617903053|        -0.24257498450062637|
| 722745| 886884423|         -0.1986432954214947|
| 990319| 790051672|        -0.09425544736694291|
| 990319| 790051672|        -0.09425544736694291|
|1423946|    497057|           4.138041806442654|
|1554865|    512980|        -0.04748441357023...|
|1725200| 987123848|        -0.01628228869020...|
|1725200| 987123848|        -0.01628228869020...|
|2305215|    491487|         0.19374196417825526|
|2418884|    515386|        -0.21774691451035974|
|2760283| 735602708|        -0.23846026399513537|
|2760283| 735602708|        -0.23846026399513537|
|2787912|    138585|        -0.14622182021013835|


In [136]:
# Recommend games for all users
game_recommendations = best_model.recommendForAllUsers(10)

In [137]:
game_recommendations.show()

+--------+--------------------+
| user_id|     recommendations|
+--------+--------------------+
|   71336|[{491870, 4.18471...|
|  507125|[{7193, 0.0}, {65...|
| 1518077|[{517795, 1.02065...|
| 2760283|[{7193, 0.0}, {65...|
| 6391627|[{7193, 0.0}, {65...|
| 7243387|[{32399, 0.700295...|
| 7313723|[{32399, 0.430283...|
| 8041705|[{7193, 0.0}, {65...|
| 8320445|[{7193, 0.0}, {65...|
| 9106364|[{7193, 0.0}, {65...|
|11308729|[{7193, 0.0}, {65...|
|17286095|[{7193, 0.0}, {65...|
|17331543|[{7193, 0.0}, {65...|
|18076161|[{7193, 0.0}, {65...|
|18583518|[{7193, 0.0}, {65...|
|18666036|[{7193, 0.0}, {65...|
|18887776|[{32982, 2.452643...|
|20386324|[{7193, 0.0}, {65...|
|20405964|[{32982, 2.881057...|
|20650721|[{32982, 2.696635...|
+--------+--------------------+
only showing top 20 rows



In [139]:
recommendation_df_final = game_recommendations.toPandas()

In [140]:
recommendation_df_final.head()

,user_id,recommendations
0,71336,"[(491870, 4.184719085693359), (32982, 3.681460..."
1,507125,"[(7193, 0.0), (6511, 0.0), (6369, 0.0), (6263,..."
2,1518077,"[(517795, 1.0206505060195923), (32982, 1.00614..."
3,2760283,"[(7193, 0.0), (6511, 0.0), (6369, 0.0), (6263,..."
4,6391627,"[(7193, 0.0), (6511, 0.0), (6369, 0.0), (6263,..."


In [184]:
game_id_to_name = pd.Series(df['game_name'].values, index=df['game_id']).to_dict()

In [185]:
recommended_dict = {}

# Assume recommendation_df_final is available and formatted correctly
for index, row in recommendation_df_final.iterrows():
    user_id = row['user_id']
    recommendations = row['recommendations']

    # Prepare the list for storing game names
    recommended_dict[user_id] = []

    # Use the dictionary to find game names and avoid DataFrame look-up
    for rec in recommendations:
        game_id = rec.game_id  # Assuming rec is an object or dict with 'game_id'
        game_name = game_id_to_name.get(game_id)  # Use dictionary to get the game name

        if game_name:  # Check if game_name exists in the dictionary
            recommended_dict[user_id].append(game_name)





In [187]:
data = []
for user_id, games in recommended_dict.items():
    for game in games:
        data.append((user_id, game))


In [188]:
# Convert the list of tuples into a DataFrame
recommended_df = pd.DataFrame(data, columns=['user_id', 'game_name'])



In [191]:
df_lists = pd.DataFrame(list(recommended_dict.items()), columns=['user_id', 'game_names'])


In [233]:
game_list = df_lists[df_lists.user_id == 27451903].game_names.values[0]

In [204]:
game_list = df_lists[df_lists.user_id == 7313723].game_names.values[0]

['Counter-Strike',
 'Grand Theft Auto V',
 'Astroneer',
 'Dishwashing Simulator',
 'Homesick',
 'Teamfight Tactics',
 'Alpha Protocol',
 'Dead by Daylight',
 'League of Legends',
 'Dota 2']

In [208]:
df[df.user_id == 27451903]

,game_id,id,language,started_at,title,type,user_id,user_name,viewer_count,user_login,game_name,tags,is_mature,box_art_url,genre,ended_at,duration_minutes,engagement_score,normalized_engagement_score,diversity_score
1358,29595,40577073669,ru,2024-04-23 13:32:48+00:00,BetBoom Streamers Battle ДЕНЬ 2 (t.me/forcaje_...,live,27451903,follow4ce,118,follow4ce,Dota 2,"['dota', 'dota2', 'чай', 'бусидо', 'Русский']",True,https://static-cdn.jtvnw.net/ttv-boxart/29595-...,Strategy,2024-04-23 18:33:40.767189+00:00,300.88,35503.84,0.001577,0.92
1359,29595,40577073669,ru,2024-04-23 13:32:48+00:00,BetBoom Streamers Battle ДЕНЬ 2 (t.me/forcaje_...,live,27451903,follow4ce,118,follow4ce,Dota 2,"['dota', 'dota2', 'чай', 'бусидо', 'Русский']",True,https://static-cdn.jtvnw.net/ttv-boxart/29595-...,MOBA,2024-04-23 18:33:40.767189+00:00,300.88,35503.84,0.001577,0.92


In [434]:
def recommend_game_streamers(user_id, df_lists, df):
    # Retrieve the list of games from df_lists for the given user_id
    try:
        game_list = df_lists[df_lists.user_id == user_id].game_names.values[0]
    except IndexError:
        return pd.DataFrame()  # Return empty DataFrame if user_id not found
    check = df[df.user_id == user_id].game_name.unique()[0]
    if check in game_list:
        return game_list.remove(check)
    if "Dishwashing Simulator" in game_list:
        game_list.remove("Dishwashing Simulator")

    print("Getting top streamers for recommended games: ", game_list)
    # Initialize an empty DataFrame to hold the streamers recommendations
    streamers_rec = pd.DataFrame()

    for game in game_list:
        # Filter the DataFrame for the current game
        x = df[df['game_name'] == game]

        # Sort the filtered DataFrame by 'viewer_count' in descending order
        x_sorted = x.sort_values(by='viewer_count', ascending=False)

        # Drop the 'genre' column and remove duplicates based on the remaining columns
        x_sorted = x_sorted.drop(columns=['genre']).drop_duplicates()

        # Get the top 3 streamers for each recommended game
        get_streamers = x_sorted.head(3)

        # Append results to the streamers_rec DataFrame
        streamers_rec = pd.concat([streamers_rec, get_streamers[['id', 'title', 'language', 'viewer_count', 'game_name']]], ignore_index=True)


    # print("Top 3 recommended streamers for each of the recommended games")

    return streamers_rec

In [422]:
df[df.user_id == 27451903]

,game_id,id,language,started_at,title,type,user_id,user_name,viewer_count,user_login,...,is_mature,box_art_url,genre,ended_at,duration_minutes,engagement_score,normalized_engagement_score,diversity_score,log_viewer_count,normalized_duration
1358,29595,40577073669,ru,2024-04-23 13:32:48+00:00,BetBoom Streamers Battle ДЕНЬ 2 (t.me/forcaje_...,live,27451903,follow4ce,118,follow4ce,...,True,https://static-cdn.jtvnw.net/ttv-boxart/29595-...,Strategy,2024-04-23 18:33:40.767189+00:00,300.88,0.346131,0.001577,0.92,4.779123,0.072426
1359,29595,40577073669,ru,2024-04-23 13:32:48+00:00,BetBoom Streamers Battle ДЕНЬ 2 (t.me/forcaje_...,live,27451903,follow4ce,118,follow4ce,...,True,https://static-cdn.jtvnw.net/ttv-boxart/29595-...,MOBA,2024-04-23 18:33:40.767189+00:00,300.88,0.346131,0.001577,0.92,4.779123,0.072426


In [252]:
df[df.user_id == 7313723]

,game_id,id,language,started_at,title,type,user_id,user_name,viewer_count,user_login,game_name,tags,is_mature,box_art_url,genre,ended_at,duration_minutes,engagement_score,normalized_engagement_score,diversity_score
162,491487,40576475685,fr,2024-04-23 10:36:05+00:00,17h Tome + Rework Haddonfield | Nerf Fléau | N...,live,7313723,ChiTai,49,chitai,Dead by Daylight,"['Fun', 'Beaujeu', 'FrogWhisperer', 'Français'...",True,https://static-cdn.jtvnw.net/ttv-boxart/491487...,Strategy,2024-04-23 18:33:40.767189+00:00,477.6,23402.4,0.001039,0.96


In [302]:
df[df.user_id == 27451903].game_name.unique()

array(['Dota 2'], dtype=object)

In [435]:
recommend_game_streamers(27451903, df_lists, df)

Getting top streamers for recommended games:  ['Grand Theft Auto V', 'Astroneer', 'Counter-Strike', 'Alpha Protocol', 'Teamfight Tactics', 'Kingdoms Reborn', 'League of Legends', 'KartRider Rush+']


,id,title,language,viewer_count,game_name
0,50947976125,MARBELLA VICE 2 | TONI GAMBINO día 2 | (de poq...,es,74372,Grand Theft Auto V
1,40577095029,JUAN DEMONIO 😈 en MARBELLA VICE II,es,18565,Grand Theft Auto V
2,41244195511,すとぐら,ja,18343,Grand Theft Auto V
3,44074114267,TUBBATHON DAY 54 - HUGE DAY - SPACE w/ @bekyamon,en,3187,Astroneer
4,50948222189,ASTRONEER WITH @tubbo I HAVE A BINGO CARD,en,901,Astroneer
5,41245055111,雑談しよう、そうしよう,ja,19,Astroneer
6,40576486853,LIVE: Astralis vs Eternal Fire - ESL Pro Leagu...,en,47429,Counter-Strike
7,42544432905,IMPERIAL VS FAZE ESL Pro League Season 19 Dia ...,pt,33245,Counter-Strike
8,50947614909,🔴2VS2 AGAINST EVERY RANK WITH @XQC🔴,en,17690,Counter-Strike
9,40577147333,"Mega Stealth Gamer™ - Alpha Protocol, First Pl...",en,8616,Alpha Protocol


In [299]:
recommended_games = x.game_name.unique().tolist()

In [305]:
game_name = df[df.user_id == 27451903].game_name.unique()[0] # For Dota2

In [395]:
def dcg_at_k(r, k, method=1):
    """Discounted cumulative gain at rank K."""
    r = np.asfarray(r)[:k]
    if r.size:
        if method == 0:
            return r[0] + np.sum(r[1:] / np.log2(np.arange(2, r.size + 1)))
        elif method == 1:
            return np.sum(r / np.log2(np.arange(2, r.size + 2)))
    return 0.

def ndcg_at_k(r, k, method=1):
    """Normalized discounted cumulative gain at rank K."""
    dcg_max = dcg_at_k(sorted(r, reverse=True), k, method)
    if not dcg_max:
        return 0.
    return dcg_at_k(r, k, method) / dcg_max

def precision_at_k(r, k):
    """Precision at rank K."""
    assert k >= 1
    r = np.asarray(r)[:k] != 0
    return np.mean(r)

def average_precision(r):
    """Average precision (AP)."""
    r = np.asarray(r) != 0
    out = [precision_at_k(r, k + 1) for k in range(r.size) if r[k]]
    if not out:
        return 0.
    return np.mean(out)

def mean_average_precision(rs):
    """Mean average precision for a list of lists of recommendations."""
    return np.mean([average_precision(r) for r in rs])

def recall_at_k(r, k, total_relevant):

    if total_relevant == 0:
        return 0  # Avoid division by zero; no relevant items to retrieve

    recall = 0
    # Ensure we do not go out of index if k is greater than the length of r
    top_k = r[:min(k, len(r))]

    for prediction in top_k:
        if prediction > 0.5:  # Assuming predictions are probabilities or scores
            recall += 1
    return recall / len(top_k)

In [396]:
recommended_games

['Grand Theft Auto V',
 'Astroneer',
 'Counter-Strike',
 'Alpha Protocol',
 'Teamfight Tactics',
 'Kingdoms Reborn',
 'Dishwashing Simulator',
 'League of Legends',
 'KartRider Rush+']

In [397]:
# For Dota 2

relevance_scores = [0.1, 0.2, 0.6, 0.1, 0.7, 0.4, 0.0, 0.9, 0.1 ]

print("NDCG@8:", ndcg_at_k(relevance_scores, 8))
print("MAP@8:", mean_average_precision([relevance_scores]))  # assuming one user
relevant_items = 0
for score in relevance_scores:
    if score > 0.5:
        relevant_items += 1

print("Recall@8:", recall_at_k(relevance_scores, 8, relevant_items))  # assuming 2 items are truly relevant

NDCG@8: 0.6358352100115684
MAP@8: 0.9704861111111112
Recall@8: 0.375


In [314]:
game_name = df[df.user_id == 7313723].game_name.unique()[0] # For Dead by Daylight

In [315]:
game_name

'Dead by Daylight'

In [436]:
recommend_game_streamers(7313723, df_lists, df)

Getting top streamers for recommended games:  ['Counter-Strike', 'Grand Theft Auto V', 'Astroneer', 'Homesick', 'Teamfight Tactics', 'Alpha Protocol', 'League of Legends', 'Dota 2']


,id,title,language,viewer_count,game_name
0,40576486853,LIVE: Astralis vs Eternal Fire - ESL Pro Leagu...,en,47429,Counter-Strike
1,42544432905,IMPERIAL VS FAZE ESL Pro League Season 19 Dia ...,pt,33245,Counter-Strike
2,50947614909,🔴2VS2 AGAINST EVERY RANK WITH @XQC🔴,en,17690,Counter-Strike
3,50947976125,MARBELLA VICE 2 | TONI GAMBINO día 2 | (de poq...,es,74372,Grand Theft Auto V
4,40577095029,JUAN DEMONIO 😈 en MARBELLA VICE II,es,18565,Grand Theft Auto V
5,41244195511,すとぐら,ja,18343,Grand Theft Auto V
6,44074114267,TUBBATHON DAY 54 - HUGE DAY - SPACE w/ @bekyamon,en,3187,Astroneer
7,50948222189,ASTRONEER WITH @tubbo I HAVE A BINGO CARD,en,901,Astroneer
8,41245055111,雑談しよう、そうしよう,ja,19,Astroneer
9,40575785429,[GTV24/7] ✶ 006 - Nachbarschaft ✶ #DOSENWURST ...,de,1429,Homesick


In [439]:
def eval_recommendation(user_id, df, df_lists, relevance_list, k):
    recommended_games = recommend_game_streamers(user_id, df_lists, df)
    game = df[df.user_id == user_id].game_name.unique()[0]
    recommeded_games = recommended_games.game_name.unique().tolist()

    if game in recommeded_games:
        recommeded_games.remove(game)
    if "Dishwashing Simulator" in recommeded_games:
        recommeded_games.remove("Dishwashing Simulator")
    print(f"Game: {game}")
    print(f"Recommended games: {recommeded_games}")
    print(f"NDCG@{k}:", round(ndcg_at_k(relevance_scores, k), 2))
    print(f"MAP@{k}:", round(mean_average_precision([relevance_scores]), 2))
    relevant_items = 0
    for score in relevance_scores:
        if score > 0.5:
            relevant_items += 1
    print(f"Recall@{k}:", round(recall_at_k(relevance_scores, k, relevant_items), 2))

In [440]:
relevance_scores = [0.1, 0.2, 0.6, 0.1, 0.7, 0.4, 0.9, 0.1 ]
eval_recommendation(27451903, df, df_lists, relevance_scores, 3)


Getting top streamers for recommended games:  ['Grand Theft Auto V', 'Astroneer', 'Counter-Strike', 'Alpha Protocol', 'Teamfight Tactics', 'Kingdoms Reborn', 'League of Legends', 'KartRider Rush+']
Game: Dota 2
Recommended games: ['Grand Theft Auto V', 'Astroneer', 'Counter-Strike', 'Alpha Protocol', 'Teamfight Tactics', 'Kingdoms Reborn', 'League of Legends', 'KartRider Rush+']
NDCG@3: 0.32
MAP@3: 1.0
Recall@3: 0.33


In [441]:
relevance_scores = [0.3, 0.6, 0.6, 0.0, 0.4, 0.1, 0.1, 0.1]

eval_recommendation(7313723, df, df_lists, relevance_scores, 3)

Getting top streamers for recommended games:  ['Counter-Strike', 'Grand Theft Auto V', 'Astroneer', 'Homesick', 'Teamfight Tactics', 'Alpha Protocol', 'League of Legends', 'Dota 2']
Game: Dead by Daylight
Recommended games: ['Counter-Strike', 'Grand Theft Auto V', 'Astroneer', 'Homesick', 'Teamfight Tactics', 'Alpha Protocol', 'League of Legends', 'Dota 2']
NDCG@3: 0.83
MAP@3: 0.91
Recall@3: 0.67
